<a href="https://colab.research.google.com/github/TsamayaDesigns/codeDivision-automation-with-python/blob/main/automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive & Set Working Directory:**

In [45]:
# 1️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# 2️⃣ Set Working Directory
import os
working_dir = '/content/drive/MyDrive/codeDivision'
os.makedirs(working_dir, exist_ok=True)
os.chdir(working_dir)

# 3️⃣ Confirm Current Working Directory
print(f"Current working directory is: {os.getcwd()}")

Mounted at /content/drive/
Current working directory is: /content/drive/MyDrive/codeDivision


**Query the Data:**

In [54]:
import re
import pandas as pd
from smtp import send
from datetime import datetime
from excel_tools import read_sheet
from creds import email, pw

fields = ['ID', 'Start_time', 'Completion_time', 'User_Email', 'Name', 'Last_modified_time', 'full_name', 'Need_365', 'Mac_or_PC', 'Moodle_Email', 'Attending', 'Alternative', 'Council', 'Postcode', 'NeuroDivergent', 'Woman', 'Minority', 'Disabled', 'Refugee']
subs1 = read_sheet('EnrolmentForm1.xlsx', fields = fields)
subs2 = read_sheet('EnrolmentForm2.xlsx', fields = fields)


**Check Attendance:**

In [55]:
# Check if person is attending
def attendance(subs_list):
  attend_y = []
  attend_n = []

  for sub in subs_list:
      if str(sub.Attending).strip().lower() in ('yes', 'y'):
          attend_y.append(sub)
      else:
          attend_n.append(sub)

  print(f'Attendance Yes: {len(attend_y)} \nAttendance No: {len(attend_n)}')
  return attend_y, attend_n

attend_y1, attend_n1 = attendance(subs1)
attend_y2, attend_n2 = attendance(subs2)


Attendance Yes: 8 
Attendance No: 9
Attendance Yes: 4 
Attendance No: 5


**Email address validation:**

In [56]:
# Check if email address is valid
EMAIL_REGEX = re.compile(
    r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
)

def validate_emails(Moodle_Email):
  valid_emails = []
  invalid_emails = []

  if not EMAIL_REGEX.match(Moodle_Email.strip()):
    invalid_emails.append(f'Moodle_Email: {Moodle_Email.strip()}')
  else:
    valid_emails.append(f'Moodle_Email: {Moodle_Email.strip()}')

  return invalid_emails, valid_emails


# Print all valid emails
def get_valid_emails(subs_list, form_name):
  # Collect all valid emails
  form_valid_emails = []

  for sub in subs_list:
      _, valid_emails = validate_emails(sub.Moodle_Email)

      if valid_emails:
        form_valid_emails.append((sub.full_name, sub.Moodle_Email))

  print(f'Valid emails from {form_name}: {form_valid_emails}')
  return form_valid_emails

valid_emails_f1 = get_valid_emails(subs1, 'EnrolmentForm1.xlsx')
valid_emails_f2 = get_valid_emails(subs2, 'EnrolmentForm2.xlsx')
print()

# Print email address report (for each person)
def email_address_report(subs_list, form_name):
  print(f'Email address report {form_name}:')
  for sub in subs_list:
      invalid_emails, valid_emails = validate_emails(sub.Moodle_Email)

      if invalid_emails:
          print(sub.full_name, 'has an invalid email:', invalid_emails)

      if valid_emails:
          print(sub.full_name, 'has a valid email:', valid_emails)

email_address_report(subs1, '(EnrolmentForm1.xlsx)')
print()
email_address_report(subs2, '(EnrolmentForm2.xlsx)')

Valid emails from EnrolmentForm1.xlsx: [('Piet Pompies', 'piet@email.com'), ('Jan Pampoen', 'jan@email.com'), ('Koos Kakebeen', 'koos@email.com'), ('Peter Pumpkineater', 'peter@email.com'), ('Gert Coetzee', 'gert@google.com'), ('Jan Pierewiet', 'JanP@outlook.com'), ('Blake Turner', 'thompsonpeter@example.org'), ('Amy Tran', 'gpacheco@example.net'), ('Anna Marsh', 'turnerstephanie@example.com'), ('William Edwards', 'patrick79@example.com'), ('Thomas Dixon', 'usims@example.net'), ('Jonathan Morton', 'deborahcuevas@example.net'), ('Russell Frederick', 'markbutler@example.org'), ('John Bowman', 'samanthabrown@example.net'), ('Alyssa White', 'millerdouglas@example.org'), ('Katrina Cook', 'leemary@example.com')]
Valid emails from EnrolmentForm2.xlsx: [('Mike Hassett', 'mike@mail.com'), ('Ben Burger', 'ben@gmail.com'), ('Clive Greenwood', 'clive@trustytransport.co.za'), ('Coolhand Luke', 'coolhand@yahoo.com'), ('Steph Rossouw', 'steph@windhoek.co.na'), ('Gert Coetzee', 'gert@google.com'), ('J

**Office 365 Requirements:**

In [57]:
# Check if student requires Office 365
def o365_count(subs_list, form_name):
  o365_y = []
  o365_n = []

  for sub in subs_list:
      if str(sub.Need_365).strip().lower() in ('yes', 'y'):
          o365_y.append(sub)
      else:
          o365_n.append(sub)

  print(f'{form_name}')
  print(f'O365 Required (Yes): {len(o365_y)} \nO365 Required (No): {len(o365_n)}')
  print()
  return o365_y, o365_n

o365_y1, o365_n1 = o365_count(subs1, 'EnrolmentForm1.xlsx')
o365_y2, o365_n2 = o365_count(subs2, 'EnrolmentForm2.xlsx')


EnrolmentForm1.xlsx
O365 Required (Yes): 7 
O365 Required (No): 10

EnrolmentForm2.xlsx
O365 Required (Yes): 5 
O365 Required (No): 4



**Available Dates:**

In [58]:
# Check alternative dates for people unable attend
for sub in subs1:
  if sub.Attending.strip().lower() in ['no', 'n']:
    try:
      date_str = sub.Alternative

      if isinstance(date_str, str):
        date = datetime.strptime(date_str.strip(), '%d/%m/%Y')
      elif isinstance(date_str, datetime):
        date = date_str
      else:
        raise ValueError("The Alternative date is not a string or datetime object.")

      print(f'{sub.full_name}, will be able to attend on: {date.strftime("%Y/%m/%d")}.')

    except ValueError:
      print(f'{sub.full_name}, entered an invalid date: \"{sub.Alternative.strip()}\", and can be contacted on {sub.Moodle_Email.strip()} to confirm alternative date(s).')


Jan Pampoen, entered an invalid date: "NA", and can be contacted on jan@email.com to confirm alternative date(s).
Koos Kakebeen, entered an invalid date: "Help", and can be contacted on koos@email.com to confirm alternative date(s).
Pieter de Coning, will be able to attend on: 2025/10/05.
Amy Tran, will be able to attend on: 2025/07/10.
Thomas Dixon, will be able to attend on: 2025/07/17.
Russell Frederick, will be able to attend on: 2025/07/22.
John Bowman, will be able to attend on: 2025/08/23.
Alyssa White, will be able to attend on: 2025/07/10.
Katrina Cook, will be able to attend on: 2025/08/04.


**Duplicate Email Check:**

In [59]:
# Duplicate email check
duplicate_emails = []
new_subs1_emails = []
new_subs2_emails = []

for full_name in subs1:
  if full_name in subs2 and full_name not in duplicate_emails:
    duplicate_emails.append(full_name)
  else:
    if full_name not in new_subs1_emails:
      new_subs1_emails.append(full_name)

for full_name in subs2:
  if full_name not in duplicate_emails and full_name not in new_subs2_emails:
    new_subs2_emails.append(full_name)

for duplicate_email in duplicate_emails:
  if len(new_subs1_emails) < len(new_subs2_emails):
    new_subs1_emails.append(duplicate_emails)
  else:
    new_subs2_emails.append(duplicate_email)

print(f'Original registrations (subs1): {len(subs1)}')
print(f'Original registrations (subs2): {len(subs2)}')
print()
print(f'Duplicate submissions (Number): {len(duplicate_emails)}')
print(f'New subs1 Emails (cohort 1): {len(new_subs1_emails)}')
print(f'New subs2 Emails (cohort 2): {len(new_subs2_emails)}')



Original registrations (subs1): 17
Original registrations (subs2): 9

Duplicate submissions (Number): 4
New subs1 Emails (cohort 1): 13
New subs2 Emails (cohort 2): 9


**Send Email to Students:**

In [66]:
# Send emails to students
def send_emails(cohort):
  for student in cohort:
    print(f'Sending email to Student: {student.ID}')

print('Sending emails to Form1 students:')
send_emails(new_subs1_emails)
print()
print('Sending emails to Form2 students:')
send_emails(new_subs2_emails)

Sending emails to Form1 students:
Sending email to Student: 1
Sending email to Student: 2
Sending email to Student: 3
Sending email to Student: 4
Sending email to Student: 5
Sending email to Student: 10
Sending email to Student: 11
Sending email to Student: 12
Sending email to Student: 13
Sending email to Student: 14
Sending email to Student: 15
Sending email to Student: 16
Sending email to Student: 17

Sending emails to Form2 students:
Sending email to Student: 1
Sending email to Student: 2
Sending email to Student: 3
Sending email to Student: 4
Sending email to Student: 5
Sending email to Student: 6
Sending email to Student: 7
Sending email to Student: 8
Sending email to Student: 9


**Sending Email to Code Division:**

In [64]:
# Prepare email body as a string
body_lines = []

body_lines.append(f'\n{len(new_subs1_emails)} Students registered as part of Cohort 1 (EnrolmentForm1.xlsx):')
for student in new_subs1_emails:
    body_lines.append(f'{student.full_name} - ({student.Moodle_Email})')

body_lines.append(f'\n{len(new_subs2_emails)} Students registered as part of Cohort 2 (EnrolmentForm2.xlsx):')
for student in new_subs2_emails:
    body_lines.append(f'{student.full_name} - ({student.Moodle_Email})')

email_body = '\n'.join(body_lines)

# Sending the Email
success = send(
    sender = email,
    app_pass = pw,
    recip = "eugene@anysupport.co.uk",
    sub = "Test Email from Colab",
    body = email_body
)

print(email_body)

Email sent successfully!

13 Students registered as part of Cohort 1 (EnrolmentForm1.xlsx):
Piet Pompies - (piet@email.com)
Jan Pampoen - (jan@email.com)
Koos Kakebeen - (koos@email.com)
Peter Pumpkineater - (peter@email.com)
Pieter de Coning - (pieteremail.com)
Anna Marsh - (turnerstephanie@example.com)
William Edwards - (patrick79@example.com)
Thomas Dixon - (usims@example.net)
Jonathan Morton - (deborahcuevas@example.net)
Russell Frederick - (markbutler@example.org)
John Bowman - (samanthabrown@example.net)
Alyssa White - (millerdouglas@example.org)
Katrina Cook - (leemary@example.com)

9 Students registered as part of Cohort 2 (EnrolmentForm2.xlsx):
Mike Hassett - (mike@mail.com)
Ben Burger - (ben@gmail.com)
Clive Greenwood - (clive@trustytransport.co.za)
Coolhand Luke - (coolhand@yahoo.com)
Steph Rossouw - (steph@windhoek.co.na)
Gert Coetzee - (gert@google.com)
Jan Pierewiet - (JanP@outlook.com)
Blake Turner - (thompsonpeter@example.org)
Amy Tran - (gpacheco@example.net)
